In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
%matplotlib inline
sns.set(style="whitegrid", palette="muted")
current_palette = sns.color_palette()
from scipy.io import loadmat
import scipy.linalg as linalg
import numpy as np
from scipy.stats import norm
from sklearn import tree
import graphviz
import sklearn.linear_model as lm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn import model_selection
from scipy import stats
import sys
sys.path.insert(0,'..\\02450Toolbox_Python\\Tools')
sys.path.insert(0,'..\\02450Toolbox_Python\\Scripts')
import similarity
import writeapriorifile



Let's load the data

In [2]:
df = pd.read_csv('OurDatabase.csv')


Let's see column names

In [3]:
df.columns

Index(['Unnamed: 0', 'Time', 'Time Code', 'Country Name', 'Country Code',
       'Fertility rate', 'Life expectancy', 'Unemployment', 'GDP per capita',
       'Population growth', 'Alcohol per capita', 'Suicide mortality rate',
       'PM2.5 air pollution', 'Population density', 'Military expenditure',
       'Terrorism Index', 'Happiness Score', 'isDevelopedEconomy',
       'Continent'],
      dtype='object')

Let's actually see the data 

In [4]:
df

,Unnamed: 0,Time,Time Code,Country Name,Country Code,Fertility rate,Life expectancy,Unemployment,GDP per capita,Population growth,Alcohol per capita,Suicide mortality rate,PM2.5 air pollution,Population density,Military expenditure,Terrorism Index,Happiness Score,isDevelopedEconomy,Continent
0,0,2015,YR2015,Afghanistan,AFG,4.802,63.298195,8.600000,599.135196,2.943234,1.0,5.5,48.016765,51.674929,0.993454,9.233,3.575,n,Asia
1,1,2015,YR2015,Albania,ALB,1.714,78.203146,17.100000,4524.990993,-0.291206,6.6,4.3,17.936625,105.135146,1.167601,2.116,4.959,n,Europe
2,2,2015,YR2015,Algeria,DZA,2.839,75.855293,11.200000,4759.595242,1.919959,0.6,3.1,35.564533,16.740504,6.274210,4.750,5.605,n,Africa
3,3,2015,YR2015,Angola,AGO,5.766,61.189341,6.200000,3730.169063,3.428021,7.6,20.5,36.395428,22.346439,3.515939,0.243,4.033,n,Africa
4,4,2015,YR2015,Argentina,ARG,2.308,76.293024,7.200000,10490.019568,1.009855,7.6,14.2,13.443965,15.865065,0.864723,1.674,6.574,n,South America
5,5,2015,YR2015,Armenia,ARM,1.622,74.206195,18.299999,3935.288772,0.368528,5.5,5.4,25.507685,102.456972,4.239226,0.115,4.350,n,Europe
6,6,2015,YR2015,Australia,AUS,1.833,82.451220,6.100000,54941.911458,1.391107,12.6,11.8,5.897735,3.096643,1.978861,3.114,7.284,d,Oceania
7,7,2015,YR2015,Austria,AUT,1.470,81.843902,5.700000,47834.787303,1.066623,8.5,16.4,17.012994,104.615307,0.664019,2.088,7.200,d,Europe
8,8,2015,YR2015,Azerbaijan,AZE,1.970,71.845195,5.000000,6117.030862,1.191210,2.1,3.3,30.354013,116.731077,5.608680,1.381,5.212,n,Europe
9,9,2015,YR2015,Bahrain,BHR,2.056,76.865195,1.100000,22436.207532,2.618665,2.4,6.5,55.454352,1779.319066,4.592714,4.871,5.960,n,Asia


In [5]:
df.index[df['Country Name'] == 'Russia']

Int64Index([98], dtype='int64')

# Creating and binarizing X

Let's get the X matrix and show shapes

In [6]:
countryNames = list(df['Country Name'])
l = list(df.columns)
attributesToShow = l[5:-2]
X = df.as_matrix(columns = attributesToShow)
N = len(df)
X.shape


(129, 12)

Let's standirdize our X matrix

In [7]:
stdVector = np.array(df[attributesToShow].std())
stdVector = np.expand_dims(stdVector,axis = 0)
X = X / stdVector
X = X - np.ones((N,1))*X.mean(0)

Let's create our binary X matrix so that there will be 2 different percentiles 

In [8]:
X_bin,attributesToShow_bin = similarity.binarize2(X,attributesToShow)

Let's show new attribute names

In [9]:
attributesToShow_bin

['Fertility rate 50th-100th percentile',
 'Life expectancy 50th-100th percentile',
 'Unemployment 50th-100th percentile',
 'GDP per capita 50th-100th percentile',
 'Population growth 50th-100th percentile',
 'Alcohol per capita 50th-100th percentile',
 'Suicide mortality rate 50th-100th percentile',
 'PM2.5 air pollution 50th-100th percentile',
 'Population density 50th-100th percentile',
 'Military expenditure 50th-100th percentile',
 'Terrorism Index 50th-100th percentile',
 'Happiness Score 50th-100th percentile',
 'Fertility rate 0th-50th percentile',
 'Life expectancy 0th-50th percentile',
 'Unemployment 0th-50th percentile',
 'GDP per capita 0th-50th percentile',
 'Population growth 0th-50th percentile',
 'Alcohol per capita 0th-50th percentile',
 'Suicide mortality rate 0th-50th percentile',
 'PM2.5 air pollution 0th-50th percentile',
 'Population density 0th-50th percentile',
 'Military expenditure 0th-50th percentile',
 'Terrorism Index 0th-50th percentile',
 'Happiness Score 

# Association Mining

Let's write apriori file 

In [10]:
writeapriorifile.WriteAprioriFile(X_bin,filename = 'CountriesApriori')

In [14]:
filename = 'CountriesApriori'
minSup = 30
minConf = 80
maxRule = 4

import numpy as np
from subprocess import run
import re
import os
import time
from sys import platform

if platform.startswith('linux'): #== "linux" or platform == "linux2":
    ext = ''  # Linux
    dir_sep = '/'
elif platform.startswith('darwin'): #== "darwin":
    ext = 'MAC'  # OS X
    dir_sep = '/'
elif platform.startswith('win'): #== "win32":
    ext = '.exe'  # Windows
    dir_sep = '\\'
else:
    raise NotImplementedError()



# Run Apriori Algorithm
print('Mining for frequent itemsets by the Apriori algorithm')
status1 = run('..{0}02450Toolbox_Python{0}Tools{0}apriori{1} -f"," -s{2} -v"[Sup. %S]" {3} apriori_temp1.txt'
              .format(dir_sep, ext, minSup, filename ), shell=True)

if status1.returncode != 0:
    print('An error occurred while calling apriori, a likely cause is that minSup was set to high such that no '
          'frequent itemsets were generated or spaces are included in the path to the apriori files.')
    exit()
if minConf > 0:
    print('Mining for associations by the Apriori algorithm')
    status2 = run('..{0}02450Toolbox_Python{0}Tools{0}apriori{1} -tr -f"," -o -n{2} -c{3} -s{4} -v"[Conf. %C,Sup. %S]" {5} apriori_temp2.txt'
                  .format(dir_sep, ext, maxRule, minConf, minSup, filename ), shell=True)

    if status2.returncode != 0:
        print('An error occurred while calling apriori')
        exit()
print('Apriori analysis done, extracting results')

# Extract information from stored files apriori_temp1.txt and apriori_temp2.txt
f = open('apriori_temp1.txt', 'r')
lines = f.readlines()
f.close()
# Extract Frequent Itemsets
FrequentItemsets = [''] * len(lines)
sup = np.zeros((len(lines), 1))
for i, line in enumerate(lines):
    FrequentItemsets[i] = line[0:-1]
    sup[i] = re.findall(' [-+]?\d*\.\d+|\d+]', line)[0][1:-1]
os.remove('apriori_temp1.txt')

# Read the file
f = open('apriori_temp2.txt', 'r')
lines = f.readlines()
f.close()
# Extract Association rules
AssocRules = [''] * len(lines)
conf = np.zeros((len(lines), 1))
for i, line in enumerate(lines):
    AssocRules[i] = line[0:-1]
    conf[i] = re.findall(' [-+]?\d*\.\d+|\d+,', line)[0][1:-1]
os.remove('apriori_temp2.txt')

# sort (FrequentItemsets by support value, AssocRules by confidence value)
AssocRulesSorted = [AssocRules[item] for item in np.argsort(conf, axis=0).ravel()]
AssocRulesSorted.reverse()
FrequentItemsetsSorted = [FrequentItemsets[item] for item in np.argsort(sup, axis=0).ravel()]
FrequentItemsetsSorted.reverse()

# Print the results
time.sleep(.5)
print('\n')
print('RESULTS:\n')
print('Frequent itemsets:')
for i, item in enumerate(FrequentItemsetsSorted):
    print('Item: {0}'.format(item))
    a = item.split('[')[0].split()
    print('Item:{}'.format(' & '.join([attributesToShow_bin[int(x)] for x in a])))
        
print('\n')
print('Association rules:')
for i, item in enumerate(AssocRulesSorted):
    print('Rule: {0}'.format(item))
    a = item.split('<-')
    sol = a[0]
    
    sag = a[1].split('[')[0] 
#     print(sag.split())
    print('{0} <- {1}'.format(' '.join([attributesToShow_bin[int(x)] for x in sol.split()]), ' & '.join([attributesToShow_bin[int(x)] for x in sag.split()])))



Mining for frequent itemsets by the Apriori algorithm
Mining for associations by the Apriori algorithm
Apriori analysis done, extracting results


RESULTS:

Frequent itemsets:
Item: 11 3[Sup. 39]
Item:Happiness Score 50th-100th percentile & GDP per capita 50th-100th percentile
Item: 6 5[Sup. 39]
Item:Suicide mortality rate 50th-100th percentile & Alcohol per capita 50th-100th percentile
Item: 13 23[Sup. 39]
Item:Life expectancy 0th-50th percentile & Happiness Score 0th-50th percentile
Item: 11 1[Sup. 38]
Item:Happiness Score 50th-100th percentile & Life expectancy 50th-100th percentile
Item: 11 12[Sup. 37]
Item:Happiness Score 50th-100th percentile & Fertility rate 0th-50th percentile
Item: 0 13 15[Sup. 37]
Item:Fertility rate 50th-100th percentile & Life expectancy 0th-50th percentile & GDP per capita 0th-50th percentile
Item: 15 17[Sup. 37]
Item:GDP per capita 0th-50th percentile & Alcohol per capita 0th-50th percentile
Item: 0 23[Sup. 37]
Item:Fertility rate 50th-100th percentile & 

So let's check which attributes are associated

In [12]:
print('{0} and {1} are associated'.format(attributesToShow_bin[0],attributesToShow_bin[13]))
print('{0} and {1} are associated'.format(attributesToShow_bin[0],attributesToShow_bin[15]))
print('{0} and {1} are associated'.format(attributesToShow_bin[1],attributesToShow_bin[12]))
print('{0} and {1} are associated'.format(attributesToShow_bin[3],attributesToShow_bin[12]))
print('{0} and {1} are associated'.format(attributesToShow_bin[15],attributesToShow_bin[13]))
print('{0} and {1} are associated'.format(attributesToShow_bin[13],attributesToShow_bin[15]))
print('{0} and {1} are associated'.format(attributesToShow_bin[12],attributesToShow_bin[16]))
print('{0} and {1} are associated'.format(attributesToShow_bin[16],attributesToShow_bin[12]))

Fertility rate 50th-100th percentile and Life expectancy 0th-50th percentile are associated
Fertility rate 50th-100th percentile and GDP per capita 0th-50th percentile are associated
Life expectancy 50th-100th percentile and Fertility rate 0th-50th percentile are associated
GDP per capita 50th-100th percentile and Fertility rate 0th-50th percentile are associated
GDP per capita 0th-50th percentile and Life expectancy 0th-50th percentile are associated
Life expectancy 0th-50th percentile and GDP per capita 0th-50th percentile are associated
Fertility rate 0th-50th percentile and Population growth 0th-50th percentile are associated
Population growth 0th-50th percentile and Fertility rate 0th-50th percentile are associated
